In [ ]:
### THIS IS THE UPDATED VERSION FOR 2024 ###

In [2]:
import pandas as pd

/var/folders/6f/h59wcfgd5r3dsh60jmr_9zg80000gn/T/ipykernel_75328/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [19]:
stats = pd.read_csv("player_mvp_stats.csv")

In [20]:
del stats["Unnamed: 0"]

In [21]:
# genuinely so confused what happened here, just deleting the duplicate row
stats[stats["Player"] == "Carnell Lake"]

,Player,Tm,Pos,Age,Games G,Games GS,Passing Cmp,Passing Att,Passing Yds,Passing TD,...,W-L%,PF,PA,PD,MoV,SoS,SRS,OSRS,DSRS,T
13181,Carnell Lake,NaN,RCB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.688,372,307,65,4.1,1.3,5.3,2.7,2.6,0


In [22]:
# remove row 36335
stats = stats.drop(36335)

In [23]:
# check for any null values
pd.isnull(stats).sum()

Player                0
Tm                   11
Pos                   0
Age                  11
Games G              11
Games GS             11
Passing Cmp          11
Passing Att          11
Passing Yds          11
Passing TD           11
Passing Int          11
Rushing Att          11
Rushing Yds          11
Rushing Y/A          11
Rushing TD           11
Receiving Tgt        11
Receiving Rec        11
Receiving Yds        11
Receiving Y/R        11
Receiving TD         11
Fumbles Fmb          11
Fumbles FL           11
Scoring TD           11
Scoring 2PM          11
Scoring 2PP          11
Fantasy FantPt     2244
Fantasy PPR        1866
Fantasy DKPt       1874
Fantasy FDPt       1873
Fantasy VBD       14903
Year                  0
Team                  0
Votes                 0
Share             16914
W                     0
L                     0
W-L%                  0
PF                    0
PA                    0
PD                    0
MoV                   0
SoS             

In [24]:
# replace NaN's w/ 0's
stats = stats.fillna(0)

In [ ]:
# w/ defensive players
predictors = ['Passing Cmp', 'Passing Att', 'Passing Yds', 'Passing TD',
       'Passing Int', 'Rushing Att', 'Rushing Yds', 'Rushing Y/A',
       'Rushing TD', 'Receiving Tgt', 'Receiving Rec', 'Receiving Yds',
       'Receiving Y/R', 'Receiving TD', 'Fumbles Fmb', 'Fumbles FL',
       'Int', 'Sk', 'Tackles Comb', 'Tackles Solo', 'Tackles Ast', 'W-L%',
       'PF', 'PA', 'PD', 'MoV', 'SRS', 'OSRS']

In [ ]:
# w/o defensive players
predictors = ['Passing Cmp', 'Passing Att', 'Passing Yds', 'Passing TD',
       'Passing Int', 'Rushing Att', 'Rushing Yds', 'Rushing Y/A',
       'Rushing TD', 'Receiving Tgt', 'Receiving Rec', 'Receiving Yds',
       'Receiving Y/R', 'Receiving TD', 'Fumbles Fmb', 'Fumbles FL',
       'W-L%', 'PF', 'PA', 'PD', 'MoV', 'SRS', 'OSRS']

In [26]:
train = stats[stats["Year"] < 2024]

In [27]:
test = stats[stats["Year"] == 2024]

In [34]:
# random forest creates series of decision trees & averages predictions from those trees
from sklearn.ensemble import RandomForestRegressor

reg = RandomForestRegressor(n_estimators=100, random_state=1, min_samples_split=5)

In [35]:
reg.fit(train[predictors], train["Share"])

RandomForestRegressor(min_samples_split=5, random_state=1)

In [36]:
predictions = reg.predict(test[predictors])

In [37]:
projected_votes = pd.DataFrame(predictions * 50, columns=["Projected MVP Votes"], index=test.index).round()

In [38]:
predictions = pd.DataFrame(predictions, columns=["Probability"], index=test.index)

In [39]:
combination = pd.concat([test[["Player", "Share", "Votes"]], predictions, projected_votes], axis=1)

In [40]:
# 2024 w/ all mvp votes & w/o defensive players
combination.sort_values("Probability", ascending=False).head(10)

,Player,Share,Votes,Probability,Projected MVP Votes
2056,Lamar Jackson,0.0,0.0,0.457280,23.0
6155,Jared Goff,0.0,0.0,0.332964,17.0
10115,Sam Darnold,0.0,0.0,0.187186,9.0
2567,Josh Allen,0.0,0.0,0.116666,6.0
13161,Saquon Barkley,0.0,0.0,0.114282,6.0
8680,Patrick Mahomes,0.0,0.0,0.064414,3.0
17028,Jayden Daniels,0.0,0.0,0.044439,2.0
4104,Joe Burrow,0.0,0.0,0.023073,1.0
6691,Jordan Love,0.0,0.0,0.020452,1.0
15981,Baker Mayfield,0.0,0.0,0.018811,1.0


In [ ]:
# 2024 w/ all mvp votes & w/ defensive players
combination.sort_values("Probability", ascending=False).head(10)

,Player,Share,Votes,Probability,Projected MVP Votes
5649,Lamar Jackson,0.0,0.0,0.461643,23.0
16978,Jared Goff,0.0,0.0,0.343562,17.0
27916,Sam Darnold,0.0,0.0,0.181040,9.0
36283,Saquon Barkley,0.0,0.0,0.131625,7.0
7083,Josh Allen,0.0,0.0,0.092443,5.0
23944,Patrick Mahomes,0.0,0.0,0.070554,4.0
46790,Jayden Daniels,0.0,0.0,0.045245,2.0
43863,Baker Mayfield,0.0,0.0,0.021343,1.0
18420,Jordan Love,0.0,0.0,0.020130,1.0
5629,Derrick Henry,0.0,0.0,0.020065,1.0


In [ ]:
combination.sort_values("Probability", ascending=False).head(10).to_csv("2024_results.csv")